In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/fashionmnist/train-images-idx3-ubyte
/kaggle/input/fashionmnist/t10k-labels-idx1-ubyte
/kaggle/input/fashionmnist/fashion-mnist_test.csv
/kaggle/input/fashionmnist/train-labels-idx1-ubyte
/kaggle/input/fashionmnist/fashion-mnist_train.csv
/kaggle/input/fashionmnist/t10k-images-idx3-ubyte


In [3]:
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_train = pd.read_csv('../input/fashionmnist/fashion-mnist_train.csv')
data_test = pd.read_csv('../input/fashionmnist/fashion-mnist_test.csv')


In [4]:
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

X = np.array(data_train.iloc[:, 1:])
y = to_categorical(np.array(data_train.iloc[:, 0]))

#Here we split validation data to optimiza classifier during training
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

#Test data
X_test = np.array(data_test.iloc[:, 1:])
y_test = to_categorical(np.array(data_test.iloc[:, 0]))



X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')
X_train /= 255
X_test /= 255
X_val /= 255

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

batch_size = 256
num_classes = 10
epochs = 100

#input image dimensions
img_rows, img_cols = 28, 28

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))


In [11]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 128)        

In [13]:
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 2s 44us/step - loss: 0.8290 - accuracy: 0.6916 - val_loss: 0.5182 - val_accuracy: 0.8143
Epoch 2/100
48000/48000 [==============================] - 2s 35us/step - loss: 0.5196 - accuracy: 0.8090 - val_loss: 0.4184 - val_accuracy: 0.8442
Epoch 3/100
48000/48000 [==============================] - 2s 34us/step - loss: 0.4425 - accuracy: 0.8401 - val_loss: 0.3707 - val_accuracy: 0.8638
Epoch 4/100
48000/48000 [==============================] - 2s 33us/step - loss: 0.3997 - accuracy: 0.8526 - val_loss: 0.3328 - val_accuracy: 0.8792
Epoch 5/100
48000/48000 [==============================] - 2s 34us/step - loss: 0.3745 - accuracy: 0.8626 - val_loss: 0.3120 - val_accuracy: 0.8854
Epoch 6/100
48000/48000 [==============================] - 2s 34us/step - loss: 0.3516 - accuracy: 0.8726 - val_loss: 0.3010 - val_accuracy: 0.8906
Epoch 7/100
48000/48000 [==============================] - 2s 

Epoch 56/100
48000/48000 [==============================] - 2s 44us/step - loss: 0.1767 - accuracy: 0.9325 - val_loss: 0.2157 - val_accuracy: 0.9206
Epoch 57/100
48000/48000 [==============================] - 2s 35us/step - loss: 0.1805 - accuracy: 0.9315 - val_loss: 0.2176 - val_accuracy: 0.9208
Epoch 58/100
48000/48000 [==============================] - 2s 34us/step - loss: 0.1776 - accuracy: 0.9317 - val_loss: 0.2176 - val_accuracy: 0.9212
Epoch 59/100
48000/48000 [==============================] - 2s 36us/step - loss: 0.1799 - accuracy: 0.9320 - val_loss: 0.2216 - val_accuracy: 0.9209
Epoch 60/100
48000/48000 [==============================] - 2s 37us/step - loss: 0.1786 - accuracy: 0.9330 - val_loss: 0.2256 - val_accuracy: 0.9171
Epoch 61/100
48000/48000 [==============================] - 2s 36us/step - loss: 0.1773 - accuracy: 0.9322 - val_loss: 0.2153 - val_accuracy: 0.9198
Epoch 62/100
48000/48000 [==============================] - 2s 33us/step - loss: 0.1781 - accuracy: 0.9325

In [14]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.20221616916283966
Test accuracy: 0.9312000274658203
